Can the reco loss be lowered by training just the decoder after pretraining with fixed encoder weights and pretraining the encoder on classification+reco? If we see an improvement is this just an effect of simply not enough epochs in the pretraining, i.e. can more training on classif+reco show the same effect of lowering the loss? Or is the decoder performance constrained by classifier performance and therefore cannot reach lower values? On the other hand can the decoder trained in isolation after pretraining reach the same loss value as training encoder+decoder in isolation? -> This would suggest that isolated training afterwards on given fixed latent space should yield optimal performance.

In [1]:
import sys; sys.path.insert(0, '..')
import utils
import keras
import numpy as np

def embed_and_translate(data, n_width, n_height):
    ndata = np.zeros((len(data), n_width, n_height, 1), dtype='float32')
    translations = np.empty((len(data), 2), dtype='float32')
    width, height = data.shape[1], data.shape[2]
    for i in range(len(data)):
        x = np.random.randint(n_width-width)
        y = np.random.randint(n_height-height)
        ndata[i][x:x+width, y:y+height] = data[i] # rows, cols = height, width
        translations[i][0] = x+(width//2)
        translations[i][1] = y+(height//2)
    return ndata, translations
            
n_splits = 16
output_shape = (56, 56, 1)
input_shape  = (14, 14, 1)
split_x, split_y = 14, 14
latent_dim = 4

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_train = x_train.astype('float32') / 255.
x_train_augmented, y_train_regr = embed_and_translate(x_train, output_shape[0], output_shape[1])
x_train_split = np.array([utils.split(x, split_x, split_y) for x in x_train_augmented], dtype='float32')
y_train = keras.utils.to_categorical(y_train)

x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
x_test = x_test.astype('float32') / 255.
x_test_augmented, y_test_regr = embed_and_translate(x_test, output_shape[0], output_shape[1])
x_test_split = np.array([utils.split(x, split_x, split_y) for x in x_test_augmented], dtype='float32')
y_test = keras.utils.to_categorical(y_test)

In [2]:
from keras.layers import Flatten, Conv2D, Conv2DTranspose, Dense, Input, Reshape, concatenate, Activation, Dropout, MaxPooling2D
from keras import Sequential
from keras.models import Model

import tensorflow as tf
import keras
import sys; sys.path.insert(0, '..')
import utils

class MultiSplit(Model):
    def __init__(self, n_splits, latent_dim, input_shape, output_shape):
        super(MultiSplit, self).__init__()
        self.encoder = self._create_encoder(latent_dim, input_shape)
        self.input_reshaper = utils.Reshaper((n_splits, *input_shape), input_shape)
        self.latent_reshaper = utils.Reshaper([latent_dim], [n_splits * latent_dim])
        self.decoder = self._create_decoder(n_splits * latent_dim, output_shape)
        self.classifier = self._create_classifier(n_splits * latent_dim)
        self.regressor = self._create_regressor(n_splits * latent_dim)
        
    def train_step(self, data):
        x, y = data
        with tf.GradientTape() as tape:
            y_pred = self(x)
            loss = self.compiled_loss(y, y_pred)
        grads = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.trainable_variables))
        self.compiled_metrics.update_state(y, y_pred)
        return {m.name: m.result() for m in self.metrics}
    
    def test_step(self, data):
        x, y = data
        y_pred = self(x, training=False)
        self.compiled_loss(y, y_pred)
        self.compiled_metrics.update_state(y, y_pred)
        return {m.name: m.result() for m in self.metrics}
    
    def call(self, x, training=True):
        z = self.encode(x, training=training)
        y_reco = self.decode(z ,training=training)
        y_class = self.classify(z ,training=training)
        y_regr = self.regress(z ,training=training)
        return {"decoder_out": y_reco, "classifier_out": y_class, "regressor_out": y_regr}
        
    def encode(self, x, training=True):
        return self.encoder( self.input_reshaper(x) , training)
    
    def decode(self, z, training=True):
        return self.decoder( self.latent_reshaper(z) , training)
    
    def classify(self, z, training=True):
        return self.classifier( self.latent_reshaper(z) , training)
    
    def regress(self, z, training=True):
        return self.regressor( self.latent_reshaper(z) , training)
    
    def _create_encoder(self, latent_dim, input_shape, n_filters=[32,64]):
        return Sequential([
            Input(shape=input_shape),
            Conv2D(filters=n_filters[0], kernel_size=(3,3), strides=(2,2), activation='relu', padding='same'),
            Conv2D(filters=n_filters[1], kernel_size=(3,3), strides=(2,2), activation='relu', padding='same'),
            Flatten(),
            Dense(units=latent_dim, activation='sigmoid')
        ], name='encoder')

    def _create_decoder(self, latent_dim, io_shape, n_filters=[32,64]):
        return Sequential([
            Input(shape=(latent_dim)),
            Dense(io_shape[0]//4 * io_shape[1]//4 * 8),  #! Reduce amount of neurons by 4.
            Reshape((io_shape[0]//4, io_shape[1]//4, 8)),
            Conv2DTranspose(filters=n_filters[1], kernel_size=(3,3), strides=(2,2), activation='relu', padding='same'),
            Conv2DTranspose(filters=n_filters[0], kernel_size=(3,3), strides=(2,2), activation='relu', padding='same'),
            Conv2DTranspose(filters=1, kernel_size=(3,3), padding='same'),
            Activation('sigmoid', name='decoder_out')
        ])
    
    def _create_classifier(self, latent_dim):
        return Sequential([
            Input(shape=(latent_dim)),
            Dense(256, activation='relu'),
            Dense(128, activation='relu'),
            Dense(64, activation='relu'),
            Dense(10, activation='softmax', name='classifier_out')
        ])
    
    def _create_regressor(self, latent_dim):
        return Sequential([
            Input(shape=(latent_dim)),
            Dense(256, activation='relu'),
            Dense(128, activation='relu'),
            Dense(64, activation='relu'),
            Dense(10, activation='relu'),
            Dense(2, activation='linear', name='regressor_out')
        ])

In [3]:
losses = {
    'decoder_out' : 'mse',
    'classifier_out' : 'categorical_crossentropy',
    'regressor_out' : 'mean_absolute_percentage_error'
}
loss_weights = {
    'decoder_out' : 1.0,
    'classifier_out' : 0.0,
    'regressor_out' : 0.0
}
model = MultiSplit(n_splits, latent_dim, input_shape, output_shape)

model.load_weights('../weights/43_CAE4x16_reco+class_epoch50_weights.h5') # MSE = 0.0062

for layer in model.encoder.layers:
    layer.trainable = False
for layer in model.regressor.layers:
    layer.trainable = False
    
model.compile(loss=losses, loss_weights=loss_weights, optimizer='adam', metrics=['accuracy'])

history = model.fit(x_train_split, {'decoder_out': x_train_augmented, 'classifier_out': y_train, 'regressor_out': y_train_regr},
                    validation_data=(x_test_split, {'decoder_out': x_test_augmented, 'classifier_out': y_test, 'regressor_out':y_test_regr}),
                    epochs=50, batch_size=32)

Epoch 1/50
1875/1875 [==============================] - 22s 12ms/step - loss: 0.0063 - classifier_out_loss: 0.4917 - decoder_out_loss: 0.0063 - regressor_out_loss: 99.8673 - classifier_out_accuracy: 0.8837 - decoder_out_accuracy: 0.9514 - regressor_out_accuracy: 0.5416 - val_loss: 0.0061 - val_classifier_out_loss: 0.4983 - val_decoder_out_loss: 0.0061 - val_regressor_out_loss: 99.8661 - val_classifier_out_accuracy: 0.8858 - val_decoder_out_accuracy: 0.9512 - val_regressor_out_accuracy: 0.5392
Epoch 2/50
1875/1875 [==============================] - 22s 12ms/step - loss: 0.0062 - classifier_out_loss: 0.4917 - decoder_out_loss: 0.0062 - regressor_out_loss: 99.8671 - classifier_out_accuracy: 0.8837 - decoder_out_accuracy: 0.9515 - regressor_out_accuracy: 0.5416 - val_loss: 0.0061 - val_classifier_out_loss: 0.4983 - val_decoder_out_loss: 0.0061 - val_regressor_out_loss: 99.8661 - val_classifier_out_accuracy: 0.8858 - val_decoder_out_accuracy: 0.9512 - val_regressor_out_accuracy: 0.5392
Epoc

Could the same decrease in val_decoder_out_loss also be achieved if we train classifier and decoder together for a few more epochs?

In [4]:
losses = {
    'decoder_out' : 'mse',
    'classifier_out' : 'categorical_crossentropy',
    'regressor_out' : 'mean_absolute_percentage_error'
}
loss_weights = {
    'decoder_out' : 1.0,
    'classifier_out' : 0.05,
    'regressor_out' : 0.0
}
model = MultiSplit(n_splits, latent_dim, input_shape, output_shape)

model.load_weights('../weights/43_CAE4x16_reco+class_epoch50_weights.h5') # MSE = 0.0062

for layer in model.encoder.layers:
    layer.trainable = False
for layer in model.regressor.layers:
    layer.trainable = False
    
model.compile(loss=losses, loss_weights=loss_weights, optimizer='adam', metrics=['accuracy'])

history = model.fit(x_train_split, {'decoder_out': x_train_augmented, 'classifier_out': y_train, 'regressor_out': y_train_regr},
                    validation_data=(x_test_split, {'decoder_out': x_test_augmented, 'classifier_out': y_test, 'regressor_out':y_test_regr}),
                    epochs=50, batch_size=32)

Epoch 1/50
1875/1875 [==============================] - 22s 12ms/step - loss: 0.0257 - classifier_out_loss: 0.3887 - decoder_out_loss: 0.0063 - regressor_out_loss: 99.8672 - classifier_out_accuracy: 0.8829 - decoder_out_accuracy: 0.9514 - regressor_out_accuracy: 0.5416 - val_loss: 0.0248 - val_classifier_out_loss: 0.3733 - val_decoder_out_loss: 0.0061 - val_regressor_out_loss: 99.8661 - val_classifier_out_accuracy: 0.8851 - val_decoder_out_accuracy: 0.9513 - val_regressor_out_accuracy: 0.5392
Epoch 2/50
1875/1875 [==============================] - 22s 12ms/step - loss: 0.0223 - classifier_out_loss: 0.3226 - decoder_out_loss: 0.0062 - regressor_out_loss: 99.8672 - classifier_out_accuracy: 0.8954 - decoder_out_accuracy: 0.9515 - regressor_out_accuracy: 0.5416 - val_loss: 0.0241 - val_classifier_out_loss: 0.3605 - val_decoder_out_loss: 0.0061 - val_regressor_out_loss: 99.8661 - val_classifier_out_accuracy: 0.8902 - val_decoder_out_accuracy: 0.9514 - val_regressor_out_accuracy: 0.5392
Epoc